# Expresiones regulares
**Autor**: Fermín Cruz   **Revisor**: Carlos G. Vallejo, J. Mariano González, José A. Troyano.  **Última modificación:** 20/12/2017

## Índice de contenidos
* [1. Información estructurada vs información no estructurada](#sec_1)
* [2. Expresiones regulares](#sec_2)
 * [2.1. Operadores básicos](#sec_2_1)
 * [2.2. Operadores avanzados](#sec_2_2)
 * [2.3. Comodines](#sec_2_3)
 * [2.4. Símbolos especiales](#sec_2_4)
* [3. Funciones para trabajar con ER en Python](#sec_3)
 * [3.1. La función re.findall](#sec_3_1)
 * [3.2. Las funciones re.match y re.fullmatch](#sec_3_2)
 * [3.3. Otras funciones que trabajan con ER](#sec_3_3)

# 1. Información estructurada vs información no estructurada <a name="sec_1"/>

En el notebook anterior hemos visto cómo procesar ficheros de texto en formato CSV. En un archivo con este formato decimos que la información está **estructurada**. Esto significa que los datos están bien ordenados, siguiendo un esquema fácil de procesar. En el caso del CSV, basta separar cada línea en trozos, utilizando la coma como separador, para obtener los distintos datos que contiene. Tal como vimos, esto puede hacerse con el método **split** de las cadenas.

Pero en otras ocasiones, la información que queremos procesar tendrá un formato más libre. A veces seguiremos teniendo información con una estructura clara, pero quizás organizada de una manera más compleja. Por ejemplo, si observamos los ficheros de *log* que genera Whatsapp para almacenar los chats, tenemos este formato:

<pre>
26/02/16, 09:16 - Leonard: De acuerdo, ¿cuál es tu punto?
26/02/16, 16:16 - Sheldon: No tiene sentido, solo creo que es una buena idea para una camiseta.
</pre>

En cada línea de estos ficheros encontramos distintos datos: la fecha y hora de un mensaje, el nombre del usuario que lo escribió, y finalmente el texto del mensaje. Pero esta información no aparece estructurada de manera tan simple como en un CSV. En este caso, la fecha aparece al principio de la línea, y acaba en una coma. A continuación viene la hora, con un espacio antes y después. Tras la hora, viene el usuario (aunque precedido de un guión y otro espacio en blanco), y finalmente, tras dos puntos y un nuevo espacio en blanco, aparece el texto del mensaje. Procesar estas líneas para extraer la información que contiene es más difícil. Ya no podemos hacer una simple llamada al método *split*. 

En otras ocasiones, la información que queremos procesar simplemente carece de estructura. Imaginemos que queremos procesar el texto de una novela, y sacar un listado de todos los nombres propios de los personajes. Decimos que el texto de la novela es información **no estructurada**. En realidad, sí que tiene una estructura, pues el texto está escrito en un determinado idioma. pero la estructura que sigue el lenguaje humano es tan compleja que, en la práctica, no podemos representarla en nuestros programas de manera exacta. Pero sí podemos hacer un acercamiento más sencillo al problema de encontrar los nombres de los personajes: extraer todas las palabras que comiencen por una letra mayúscula. Con esto obtendríamos un conjunto que incluiría sin duda los nombres de los personajes (aunque luego tuviéramos que ingeniar algún procedimiento posterior para filtrar las que no corresponden a nombres de personajes).

En este tipo de situaciones, cuando analizamos información con una estructura compleja o directamente no estructurada, tenemos a nuestra disposición un recurso de gran potencia: las **expresiones regulares**. 

# 2. Expresiones regulares <a name="sec_2"/>

Una **expresión regular** (que podemos abreviar como **ER**, o **RE** en inglés, por *regular expression*) es una cadena de texto que define un patrón, de manera que dado otro texto podemos decir si existe o no una coincidencia con el patrón definido por dicha expresión regular. Lo entenderás mejor si vemos algunos ejemplos. Antes de ver cómo se escriben las expresiones regulares, vamos a escribirlas en lenguaje natural.

Una posible ER podría ser *"todas las palabras que comiencen por mayúsculas"*. La **expresión regular define un patrón en la que una cadena determinada puede o no encajar**. Por ejemplo, la cadena *"casa"* no encaja en el patrón (pues no empieza por letra mayúscula). Tampoco encajaría la cadena "1298,3", pues no es una palabra (entendemos que cuando se habla de *todas las palabras* se refiere a combinaciones de letras). Sin embargo, la cadena *"Tomás"* sí encaja en el patrón. 

Una vez definida una ER, no sólo podremos usarla para decidir si una cadena la cumple o no. **También podremos aplicarla a cadenas más largas y pedirle a Python que nos devuelva todos los trozos de la cadena que cumplen dicho patrón**. Siguiendo con el ejemplo anterior, podríamos procesar la cadena *"Tomás salió de casa y se encontró con Simón."*, y Python nos devolvería una lista con los trozos que cumplen el patrón, en este caso *['Tomás', 'Simón']*.

Y no sólo eso: también podemos definir patrones más complejos formados por distintos grupos, aplicar dicho patrón a una cadena de entrada y obtener distintos trozos de esa cadena como salida. Por ejemplo, podremos definir un patrón que, aplicado a una línea de un log de Whatsapp como el que vimos antes, nos devuelva por separado los trozos correspondientes a la fecha, la hora, el usuario y el texto del mensaje. 

Aunque lo que vamos a estudiar aquí es la sintaxis propia de Python para la definición de expresiones regulares, dicha sintaxis es un estándar *de facto*, lo que significa que podremos usarla tal cual o de forma muy parecida en otros contextos (por ejemplo, programando en otros lenguajes como Java, o haciendo búsquedas en editores de texto avanzados como *Notepad++* o *Sublime Text*, entre otros).

Para poder ejecutar los ejemplos que vayamos viendo, veamos un trozo de código que aplica una ER a una cadena y nos dice si la cadena cumple o no el patrón:

In [ ]:
# Para trabajar con expresiones regulares, hay que hacer esta importación:
import re  

def prueba_ER(expresion_regular):
    print('Probando la ER "' + expresion_regular + '"')
    cadena = input("Introduce una cadena:")    
    if re.fullmatch(expresion_regular, cadena):
        print('La cadena ENCAJA en el patrón.')
    else:
        print('La cadena NO ENCAJA en el patrón.')

prueba_ER(r'Tomás')

La función **re.fullmatch** recibe una expresión regular y una cadena. Si hay coincidencia entre la cadena y la expresión regular, devuelve un objeto de tipo **match**. Si no hay coincidencia, devuelve *None*. Más adelante veremos qué es este objeto *match*; por ahora nos basta saber que si ha habido coincidencia, en el código anterior se ejecutará el *if*, y si no la ha habido, se ejecutará el *else*.

Aunque no es obligatorio, es habitual poner una erre (r) delante de las cadenas que expresan expresiones regulares (veremos más adelante por qué). La expresión regular que hemos usado en el ejemplo anterior es del tipo más simple posible: la única cadena que encaja con el patrón es *"Tomás"*. Vamos a ir complicándola poco a poco, introduciendo distintos operadores que nos permitirán expresar patrones más complejos.

## 2.1. Operadores básicos <a name="sec_2_1"/>

Existen tres operadores básicos: concatenación, unión y repetición.  

La **concatenación** se consigue simplemente escribiendo expresiones regulares una a continuación de la otra. Por ejemplo, la ER ```r'Tomás'``` es el resultado de concatenar las ER ```r'T'```, ```r'o'```, ```r'm'```, ```r'á'``` y ```r's'```. Cada una de estas expresiones encajan únicamente con los caracteres correspondientes. Con la concatenación obtenemos una nueva ER que encaja con aquellas cadenas formadas por subcadenas que encajen cada una de ellas con cada una de las ER anteriores. 

La **unión** se consigue mediante el operador **|**. Si unimos dos ER mediante este operador, la nueva ER encaja con aquellas cadenas que encajen con cualquiera de las dos ER anteriores. Prueba a introducir en el siguiente ejemplo las cadenas *"Tomás"* y *"Simón"* para comprobarlo:

In [ ]:
prueba_ER(r'Tomás|Simón')

La **repetición** se obtiene mediante los operadores \* y *+*. El operador se coloca a continuación de una ER, de manera que las cadenas que encajarán serán aquellas formadas por cualquier número de repeticiones de cadenas que encajen con la ER a la que se le aplicó el operador de repetición. La diferencia entre \* y *+* es que el primer operador contempla la posibilidad de cero repeticiones (es decir, admite la cadena vacía como coincidencia). Prueba el siguiente ejemplo:

In [ ]:
prueba_ER(r'a*') # Admite cualquier cadena formada por cualquier número de aes, incluyendo la cadena vacía.
prueba_ER(r'a+') # Admite cualquier cadena formada por una o más aes.

Combinando estos tres operadores podemos expresar patrones más complejos. Para dejar claro el alcance de cada operador, puede ser útil utilizar paréntesis. Por ejemplo, *"cadenas que comiencen por una a y, a continuación, tengan cualquier combinación de aes y bes"*. Puedes probar el siguiente ejemplo escribiendo cadenas como *"aaab"* o *"abbaabbbababaa"*:

In [ ]:
prueba_ER(r'a(a|b)*')

## 2.2. Operadores avanzados <a name="sec_2_2"/>

Aunque haciendo uso de los operadores anteriores es posible escribir cualquier expresión regular, en muchos casos sería necesario escribir patrones demasiado largos. Existen una serie de operadores "avanzados" que nos permiten abreviar ciertos usos comunes de los operadores básicos.

El operador **[ ]** permite expresar la unión entre un conjunto de caracteres. La ER resultante identifica cadenas formadas por uno cualquiera de los caracteres incluidos entre los corchetes. Por ejemplo, el patrón ```r'a[abcde]*'``` se puede leer como *"cualquier cadena que comience por una a y, a continuación, tenga cualquier combinación de aes, bes, ces, des y es"*. Date cuenta de que podríamos haber escrito la misma expresión usando los operadores básicos, aunque nos saldría algo más larga y un poco menos legible: ```r'a(a|b|c|d|e)*'```.

In [ ]:
prueba_ER(r'a([abcde])*')

Dentro de los corchetes, podemos expresar intervalos de caracteres de forma aún más abreviada, utilizando el guión. Por ejemplo, la expresión anterior también se puede escribir así: ```r'a[a-e]*```. Esto es especialmente útil cuando queremos expresar cosas como *"cualquier caracter en letras minúsculas"*, que podremos escribir como ```[a-z]```. Mira el siguiente ejemplo:

In [ ]:
prueba_ER(r'[0-9]+') # Encaja con cualquier combinación de dígitos

También es posible indicar varios intervalos de caracteres, de la siguiente manera:

In [ ]:
prueba_ER(r'[0-9a-zA-Z]+') # Encaja con cualquier combinación de dígitos y letras

Para acabar con el operador *[ ]*, si colocamos el carácter *^* a continuación del corchete de apertura, estaremos invirtiendo la selección de caracteres, por ejemplo:

In [ ]:
prueba_ER(r'[^0-9]+') # Encaja con cualquier combinación de caracteres que no incluya ningún dígito

---
El operador **?** expresa que una parte del patrón es opcional. Dicho de otro modo, permite cero o una repetición de la ER a la que afecta. Por ejemplo, el siguiente patrón coincide con la palabra "casa", en singular o en plural:

In [ ]:
prueba_ER(r'casas?')

Fíjate en que la interrogación sólo afecta al último carácter. Si se desea expresar opcionalidad para un trozo más grande del patrón, hay que utilizar los paréntesis:

In [ ]:
prueba_ER(r'(muy )?bien') # Encaja con "bien" y con "muy bien"

---
El operador **{ }** es parecido a los operadores de repetición, con la salvedad de que permite escoger el número exacto de repeticiones. Por ejemplo, el patrón ```r'[0-9]{10}'``` coincide con cadenas formadas *exactamente* por 10 dígitos:

In [ ]:
prueba_ER(r'[0-9]{10}')

También podemos especificar intervalos permitidos de repeticiones:

In [ ]:
prueba_ER(r'[0-9]{1,10}') # Entre uno y diez dígitos

In [ ]:
prueba_ER(r'[0-9]{,10}') # Entre cero y diez dígitos

In [ ]:
prueba_ER(r'[0-9]{3,}') # Al menos tres dígitos

## 2.3. Comodines <a name="sec_2_3"/>

Ya que es frecuente usar en una ER el patrón *"cualquier dígito"*, es posible expresarlo de una manera más concisa mediante la expresión ```\d```. Así que podemos volver a escribir *"cadenas formadas exactamente por 10 dígitos"* de esta otra forma:

In [ ]:
prueba_ER(r'\d{10}')

Decimos que ```\d``` es un **comodín** (también llamado *clase de caracteres*). Existen algunas más, que enumeramos a continuación:

|  Comodín  | Significado         |
|------|-----------------|
|.| Cualquier carácter, excepto el salto de línea.|
|\d| Cualquier dígito. |
|\D| Cualquier carácter distinto de un dígito.|
|\w| Cualquier carácter que puede formar parte de una palabra, incluyendo dígitos, letras y el carácter subrayado (\_).|
|\W| Cualquier carácter distinto a los expresados por \w.|
|\s| Cualquier tipo de espacio en blanco, incluyendo la barra espaciadora, el tabulador y el salto de línea. |
|\S| Cualquier carácter distinto a los expresados por \s.|
|\b| Representa el borde de una palabra.|
|^| Representa el comienzo de una cadena.|
|$| Representa el final de una cadena.|

Los tres últimos comodines, ```\b```, ```^``` y ```$```, no encajan realmente con ningún carácter, sino que sirven para representar distintas situaciones. Por ejemplo, para expresar en una ER que la cadena debe comenzar por un número, podríamos escribir ```^\d```.  Y la expresión ```\b\d*``` aplicada a la cadena *"casa231 365"* encarajaría con el trozo *"231"* (pues es una combinación de números que aparece a continuación de una palabra), pero no con el trozo *"365"* (en la sección 3.1 veremos otro ejemplo de uso del comodín ```\b```).

Échale un ojo a los siguientes ejemplos en los que se usan comodines:

In [ ]:
# Encaja con cualquier cadena que comience por una a
prueba_ER(r'a.*')

In [ ]:
# Encaja con dos palabras cualesquiera y un número de uno o varios dígitos
# Por ejemplo: Me debes 120
prueba_ER(r'\w+\s\w+\s\d+')

In [ ]:
# Encaja con una fecha como las siguientes:
# 1/1/2018
# 12/3/18
# 25/12/17
prueba_ER(r'\d?\d/\d?\d/(\d\d)?\d\d')

La mayoría de los comodines comienzan con una barra invertida. Esta es la razón por la que las expresiones regulares suelen escribirse en Python en cadenas antecedidas por una erre. Si no ponemos la erre, y hacemos uso de una barra invertida dentro de la cadena, Python interpreta erróneamente que queremos introducir un carácter especial en la cadena (como el \n para el salto de línea, o el \t para el tabulador). Sin embargo, al anteceder la cadena con una erre, Python interpreta las barras invertidas de manera literal; si no pusiéramos la erre, habría que usar dos barras invertidas (\\\\) cada vez que quisiéramos indicar un comodín. 

## 2.4. Símbolos especiales <a name="sec_2_4"/>

En las secciones anteriores hemos visto que algunos caracteres tienen un significado especial cuando aparecen en una expresión regular: el punto, el asterisco, la interrogación, los corchetes... ¿Cómo hacemos entonces para utilizar esos caracteres de manera literal en nuestras expresiones regulares? Por ejemplo, supongamos que queremos escribir una ER que signifique *"palabra acabada en interrogación"*. 

La manera de hacerlo es "escapar" los caracteres especiales, lo que significa ponerles delante una barra invertida (\\). De esta forma, el carácter en cuestión deja de tener un significado especial, y se interpreta de manera literal. 

Por tanto, la expresión *"palabra acabada en interrogación"* quedaría así: ```r'\w+\?'```.

# 3. Funciones para trabajar con ER en Python <a name="sec_3"/>

Aunque existen [muchas más funciones en el módulo re](https://docs.python.org/3/library/re.html), nosotros nos limitaremos a ver tres que nos pueden resultar muy útiles a la hora de procesar ficheros de texto con estructura compleja, como el ejemplo que vimos anteriormente de los log de Whatsapp. En concreto, vamos a estudiar las siguientes funciones:

* **re.findall**: recibe una expresión regular y una cadena y devuelve una lista con todos los trozos de la cadena que encajan con el patrón.

* **re.match**: recibe una expresión regular y una cadena, y comprueba si al principio de dicha cadena hay algún trozo que coincida con la ER (puede ser la cadena completa o sólo una parte). 

* **re.fullmatch**: recibe una expresión regular y una cadena, y comprueba si la cadena completa coincide con el patrón. 

## 3.1. La función re.findall <a name="sec_3_1"/>

La función **re.findall** recibe una expresión regular y una cadena y devuelve una lista con todos los trozos de la cadena que encajan con el patrón. Observa el siguiente ejemplo, en el que buscamos todas las palabras que acaben en tilde y contamos cuántas veces aparecen:

In [ ]:
from collections import Counter

# El fichero alicia.txt contiene el texto de la novela "Alicia en el País de las Maravillas"
contador = Counter()
with open('alicia.txt', encoding='utf-8') as f:
    for linea in f:        
        linea = linea.lower() # Pasamos todo el texto a minúsculas
        acaban_en_tilde = re.findall(r'\w*[áéíóú]\b', linea)
        # Le pasamos la lista de palabras encontradas al objeto Counter,
        # que se encarga de ir contando cuántas veces aparece cada palabra.
        contador.update(acaban_en_tilde)
    # El método most_common del contador nos devuelve una lista de tuplas con
    # las palabras y las veces que han aparecido cada una
    print(contador.most_common())

El patrón que hemos utilizado, ```r'\w*[áéíóú]\b'```, significa *"cualquier palabra acabada en á, é, í, ó o ú"*. Fíjese en el uso de ```\b``` para representar el borde de una palabra (prueba a quitar ese ```\b``` del patrón y mira lo que ocurre).

Si la expresión regular que utilizamos contiene paréntesis, la función *re.findall* se comporta de una manera un poco distinta. Observa el siguiente ejemplo:

In [ ]:
siguiente_alicia = []
with open('alicia.txt', encoding='utf-8') as f:
    for linea in f:        
        siguiente_alicia += re.findall(r'Alicia (\w+)', linea)
    print(sorted(set(siguiente_alicia)))

En este código estamos buscando todas las apariciones de *"Alicia"* seguida de cualquier palabra. Pero cuando se encuentra un trozo de texto que encaja con este patrón, *findall* nos devuelve únicamente la parte que corresponde a lo que hemos metido dentro de los paréntesis. ¿Y si hay más de una pareja de paréntesis en nuestra expresión regular? Compruébalo tú mismo:

In [ ]:
siguiente_alicia = []
with open('alicia.txt', encoding='utf-8') as f:
    for linea in f:        
        siguiente_alicia += re.findall(r'Alicia (\w+) (\w+)', linea)
    print(sorted(set(siguiente_alicia)))

Efectivamente: si hay varios paréntesis, *findall* nos devuelve tuplas, con los trozos de cadena correspondientes a cada uno de los paréntesis. Esto puede ser muy útil para extraer determinados datos desde un texto. Por ejemplo, podemos extraer los números y nombres de los capítulos del cuento:

In [ ]:
capitulos = []
with open('alicia.txt', encoding='utf-8') as f:
    for linea in f:        
        capitulos += re.findall(r'Capítulo (\d+) - (.*)', linea)
    print(capitulos)

## 3.2. Las funciones re.match y re.fullmatch <a name="sec_3_2"/>

La función **re.match** recibe una ER y una cadena, y comprueba si **al principio de dicha cadena** hay algún trozo que coincida con la ER (puede ser la cadena completa o sólo una parte). Si se encuentra una coincidencia, se obtiene un objeto **match**, el cual nos permite consultar el trozo concreto de cadena que se ha encontrado conforme al patrón. 

Mira el siguiente ejemplo: 

In [ ]:
with open('bigbangtheory_es.txt', encoding='utf-8') as f:
    for linea in f:        
        print('Línea que se va a procesar:', linea)
        coincidencia = re.match(r'\d\d?/\d\d?/\d\d?', linea)
        if coincidencia:
            print('Fecha encontrada:', coincidencia.group())
            # Para este ejemplo, abortamos el recorrido del fichero una vez
            # hemos encontrado la primera fecha.
            break;

El código anterior busca en cada línea un patrón que corresponde con una fecha. Si la función *re.match* encuentra una coincidencia al principio de la cadena, entonces el *if* lo evalúa como *True*. Una vez dentro del *if*, el método **group** nos devuelve el trozo de cadena correspondiente a dicha coincidencia. 

El funcionamiento de *re.match* es muy parecido al de la función **re.fullmatch** que vimos al principio de este notebook, con la única diferencia de que *re.fullmatch* sólo encuentra coincidencia si la cadena **completa** coincide con la ER. Eso la hace útil para probar expresiones regulares que estamos desarrollando. Si en el ejemplo anterior hubiéramos usado *re.fullmatch*, no se habría encontrado ninguna coincidencia, ya que el patrón se da al principio de la línea, pero no coincide con la cadena completa (pruébalo cambiando el código de arriba).La función *re.fullmatch* también puede sernos útil para validar que un dato introducido por el usuario de nuestra aplicación sigue un formato concreto (la típica celda de un formulario que *te obliga* a introducir una fecha correctamente). Mira el siguiente ejemplo:

In [ ]:
er_fecha = r'\d?\d/\d?\d/(\d\d)?\d\d'
fecha = input('Introduzca una fecha:')
while not re.fullmatch(er_fecha, fecha):
    fecha = input('Formato incorrecto, debe introducir una fecha con formato día/mes/año:')
print('Fecha correctamente introducida:', fecha)

### ¡Prueba tú!
Completa el siguiente código para que obligue al usuario a introducir una dirección de email válida. Las direcciones de email están formadas por el nombre de usuario (que simplificaremos diciendo que sólo admite letras, dígitos y el signo de subrayado), una arroba, y el nombre del dominio del servidor de correo (formado por una combinación de letras, un punto, y otra combinación de letras). Por ejemplo, sería válido *"tomas@fp.com"*, pero no serían válidos *"tomas"*, *"@fp,com"*, ni "tomas@fp".

In [ ]:
er_email = r'COMPLETAR'
email = input('Introduzca una dirección de email:')
while not re.fullmatch(er_email, email):
    email = input('Formato incorrecto, debe introducir una dirección de email con formato usuario@servidor:')
print('Email correctamente introducido:', email)

---
Al igual que ocurre con la función *re.findall*, si la expresión regular contiene paréntesis estos también tienen un efecto sobre el comportamiento de las funciones *re.match* y *re.fullmatch*. Cada una de las parejas de paréntesis representan un grupo, de manera que podemos acceder a cada uno de los grupos por separado, indicándolo mediante un número que le pasamos como parámetro al método *group*:

In [ ]:
with open('bigbangtheory_es.txt', encoding='utf-8') as f:
    for linea in f:        
        coincidencia = re.match(r'(\d\d?)/(\d\d?)/(\d\d?)', linea)
        if coincidencia:
            dia = coincidencia.group(1)
            mes = coincidencia.group(2)
            anyo = coincidencia.group(3)
            print('Fecha encontrada: dia {}, mes {}, año {}'.format(dia, mes, anyo))
            # Para este ejemplo, abortamos el recorrido del fichero una vez
            # hemos encontrado la primera fecha.
            break;

Podemos mejorar la legibilidad de la expresión regular y del código que accede a los grupos si ponemos nombre a dichos grupos. Esto se puede hacer escribiendo los grupos de esta forma: ```(?P<nombre> ...)```. Una vez hecho esto, podemos pasar la cadena con el nombre de un grupo al método *group*, en lugar de un número. Veamos cómo queda el ejemplo anterior si nombramos los grupos:

In [ ]:
with open('bigbangtheory_es.txt', encoding='utf-8') as f:
    for linea in f:        
        coincidencia = re.match(r'(?P<dia>\d\d?)/(?P<mes>\d\d?)/(?P<año>\d\d?,)', linea)
        if coincidencia:
            dia = coincidencia.group('dia')
            mes = coincidencia.group('mes')
            anyo = coincidencia.group('año')
            print('Fecha encontrada: dia {}, mes {}, año {}'.format(dia, mes, anyo))
            # Para este ejemplo, abortamos el recorrido del fichero una vez
            # hemos encontrado la primera fecha.
            break;

Si completamos la expresión regular que estamos usando, podremos extraer todos los trozos que nos interesan de cada una de las líneas del fichero de log. Vamos a definir grupos dentro de la ER para extraer la fecha, la hora, el usuario y el texto del mensaje:

In [ ]:
with open('bigbangtheory_es.txt', encoding='utf-8') as f:
    for linea in f:        
        coincidencia = re.match(r'(?P<fecha>\d\d?/\d\d?/\d\d?), (?P<hora>\d\d:\d\d) - (?P<usuario>[^:]+):(?P<texto>.*)', linea)
        if coincidencia:
            fecha = coincidencia.group('fecha')
            hora = coincidencia.group('hora')
            usuario = coincidencia.group('usuario')
            texto = coincidencia.group('texto')
            print('Fecha: {}. Hora: {}. Usuario: {}.'.format(fecha, hora, usuario))
            print('Texto:', texto)
            # Para este ejemplo, abortamos el recorrido del fichero una vez
            # hemos encontrado la primera fecha.
            break;

Ten en cuenta que todos los trozos extraídos son cadenas; es posible que fuese apropiado convertir algunas de estas cadenas a otros tipos de datos que representen mejor la información (por ejemplo, la fecha a tipo *date* y la hora a tipo *time*).

## 3.3. Otras funciones que trabajan con ER <a name="sec_3_3"/>

Aunque no las veremos en detalle, aquí tienes otras funciones existentes en el módulo *re* que alguna vez podrían serte útiles:
* **re.search**: muy parecida a *re.match*, con la única diferencia de que busca una coincidencia **en cualquier parte de la cadena**, no necesariamente al principio.
* **re.split**: recibe una expresión regular y una cadena. Al igual que el método *split* de las cadenas, devuelve una lista que corresponde a la cadena troceada. La diferencia con el método *split* es que se usa como separador una expresión regular, en lugar de una cadena literal. Así que, por ejemplo, podríamos dividir una cadena usando comas, puntos o puntos y coma como separadores, indistintamente, usando la ER ```r'[,.;]'```.
* **re.sub**: recibe una expresión regular, una cadena de sustitución y una cadena a procesar. Busca todas las coincidencias de la ER dentro de la cadena a procesar, y las sustituye por la cadena de sustitución, devolviendo la cadena resultante. 